dieses Notebook testet die Logik für das Script 00_create_sun_reference.py 

In [1]:
import sys
import pandas as pd
from pathlib import Path
from datetime import date, timedelta
from astral import LocationInfo
from astral.sun import sun

In [9]:
from zoneinfo import ZoneInfo
sys.path.append(str(Path().cwd().parent))

from scripts.utils import load_config, setup_logger

In [11]:
config_path = "../config/pipeline.yaml"
cfg = load_config(config_path)
lat = cfg["location"]["latitude"]
lon = cfg["location"]["longitude"]

In [16]:
# 2. Standort & Zeitzone definieren
    # Wir nutzen explizit die IANA Zeitzone für Deutschland
berlin_tz = ZoneInfo("Europe/Berlin")
city = LocationInfo(name="Biotop", region="Germany",  timezone="Europe/Berlin", latitude=lat, longitude=lon)

In [22]:
city, city.observer

(LocationInfo(name='Biotop', region='Germany', timezone='Europe/Berlin', latitude=51.32, longitude=6.78),
 Observer(latitude=51.32, longitude=6.78, elevation=0.0))

In [18]:
berlin_tz

zoneinfo.ZoneInfo(key='Europe/Berlin')

In [48]:
start_date = date(2025, 1, 1)
end_date = date(2025, 12, 31)
current_date = date(2025, 3,30)

In [49]:
start_date, end_date

(datetime.date(2025, 1, 1), datetime.date(2025, 12, 31))

In [84]:
current_date.month

3

In [50]:
s = sun(city.observer, date=current_date, tzinfo=berlin_tz)
s

{'dawn': datetime.datetime(2025, 3, 30, 6, 38, 37, 233706, tzinfo=zoneinfo.ZoneInfo(key='Europe/Berlin')),
 'sunrise': datetime.datetime(2025, 3, 30, 7, 12, 52, 61849, tzinfo=zoneinfo.ZoneInfo(key='Europe/Berlin')),
 'noon': datetime.datetime(2025, 3, 30, 13, 37, 23, tzinfo=zoneinfo.ZoneInfo(key='Europe/Berlin')),
 'sunset': datetime.datetime(2025, 3, 30, 20, 2, 39, 812064, tzinfo=zoneinfo.ZoneInfo(key='Europe/Berlin')),
 'dusk': datetime.datetime(2025, 3, 30, 20, 37, 2, 752007, tzinfo=zoneinfo.ZoneInfo(key='Europe/Berlin'))}

In [51]:
# B) Werte extrahieren
sr_aware = s["sunrise"]
ss_aware = s["sunset"]

In [87]:
print(sr_aware,'\n',ss_aware)
print(type(sr_aware))

2025-03-30 07:12:52.061849+02:00 
 2025-03-30 20:02:39.812064+02:00
<class 'datetime.datetime'>


In [93]:
sr_aware.month

3

wenn wir für ein Datum den Sonnenaufgang/untergang extrahieren, wird die Uhrzeit korrekt als datetime Objekt gespeichert, im richtigen Schema

In [95]:
sr_naive = sr_aware.replace(tzinfo=None)
ss_naive = ss_aware.replace(tzinfo=None)
print(sr_naive, '\n',ss_naive)
print(type(sr_naive))

2025-03-30 07:12:52.061849 
 2025-03-30 20:02:39.812064
<class 'datetime.datetime'>


In [97]:
print(sr_naive.year)
print(sr_naive.month)
print(sr_naive.day)
print(sr_naive.hour)
print(sr_naive.minute)
print(sr_naive.second)
print(sr_naive.microsecond)

2025
3
30
7
12
52
61849


die Naive Zeit ist auch korrekt und ein datetime Objekt

In [54]:
rows = []
rows.append({
                "date": current_date,
                # 1. Für die Wissenschaft (Aware ISO-String)
                "sunrise_aware": sr_aware.isoformat(),
                "sunset_aware": ss_aware.isoformat(),
                
                # 2. Für das Inventory-Matching (Naive Timestamp)
                "sunrise_naive": sr_naive,
                "sunset_naive": ss_naive,
                
                # 3. Metadaten
                "noon_naive": s["noon"].replace(tzinfo=None),
                "dst_active": bool(sr_aware.dst()) # True wenn Sommerzeit
})

In [55]:
rows

[{'date': datetime.date(2025, 3, 30),
  'sunrise_aware': '2025-03-30T07:12:52.061849+02:00',
  'sunset_aware': '2025-03-30T20:02:39.812064+02:00',
  'sunrise_naive': datetime.datetime(2025, 3, 30, 7, 12, 52, 61849),
  'sunset_naive': datetime.datetime(2025, 3, 30, 20, 2, 39, 812064),
  'noon_naive': datetime.datetime(2025, 3, 30, 13, 37, 23),
  'dst_active': True}]

bis hier hin hat der Code einwandfrei funktioniert. Jetzt wird die erzeugte Tabelle untersucht

In [59]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
sonne = pd.read_csv("../outputs/reference_sun.csv")

---

# Bemekungen zu csv